Import required libraries


In [8]:
!pip install imutils


  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=5cf0e3aca91684285114b3c01714b30a38db6c4578e76c7211380730db2c8fba
  Stored in directory: c:\users\jagad\appdata\local\pip\cache\wheels\59\1b\52\0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89
Successfully built imutils


In [7]:
conda-h

usage: conda-script.py [-h] [-V] command ...

conda is a tool for managing and deploying applications, environments and packages.

Options:

positional arguments:
  command
    clean        Remove unused packages and caches.
    compare      Compare packages between conda environments.

Note: you may need to restart the kernel to use updated packages.    config       Modify configuration values in .condarc. This is modeled

                 after the git config command. Writes to the user .condarc
                 file (C:\Users\jagad\.condarc) by default.
    create       Create a new conda environment from a list of specified
                 packages.
    help         Displays a list of available conda commands and their help
                 strings.
    info         Display information about current conda install.
    init         Initialize conda for shell interaction. [Experimental]
    install      Installs a list of packages into a specified conda
                 environment.

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os



# initialize the initial learning rate, number of epochs to train for,
# and batch size
#initial learning rate 
INIT_LR = 1e-4
#number of epochs
EPOCHS = 15
#batch size
BS = 32

dir=r"E:/face_mask/Face-Mask-Detection-master/dataset"
#in the dataset we have two categories of data so......
categories=["with_mask","without_mask"]


#lets create two empty lists
data = []
labels = []

#iterate through images
for category in categories:
    path = os.path.join(dir, category)
    #iterate through list of images
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        #create an array of images
        image = img_to_array(image)
        #preprocess the data
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)
        #checkpoint
#perform encoding on labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
#checkpoint
data = np.array(data, dtype="float32")
labels = np.array(labels)

#split the data into train and test
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42) #80% for training
#perform some operations on training images

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,    #<tensorflow.python.keras.preprocessing.image.ImageDataGenerator object at 0x000001F470FB7070>
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")
#checkpoint

#load the MobileNetV2 network

base_model = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(128, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation="softmax")(head_model)
#checkpoint

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=base_model.input, outputs=head_model)


# loop over all layers in the base model and freeze them so they will*not* be updated during the first training process
for layer in base_model.layers:
    layer.trainable = False

    
#compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

# train the head of the network

print("[INFO] training head...")
H = model.fit(
    aug.flow(train_x, train_y, batch_size=BS),
    steps_per_epoch=len(train_x) // BS,
    validation_data=(test_x, test_y),
    validation_steps=len(test_x) // BS,
    epochs=EPOCHS)

print("[INFO] evaluating network...")
predIdxs = model.predict(test_x, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(test_y.argmax(axis=1), predIdxs,target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detect.model", save_format="h5")






E:\New folder\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


E:\New folder\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


[INFO] training head...
Epoch 1/15
95/95 [==============================] - 142s 1s/step - loss: 0.4186 - accuracy: 0.8411 - val_loss: 0.1474 - val_accuracy: 0.9817
Epoch 2/15
95/95 [==============================] - 121s 1s/step - loss: 0.1366 - accuracy: 0.9697 - val_loss: 0.0751 - val_accuracy: 0.9909
Epoch 3/15
95/95 [==============================] - 129s 1s/step - loss: 0.0976 - accuracy: 0.9720 - val_loss: 0.0535 - val_accuracy: 0.9896
Epoch 4/15
95/95 [==============================] - 126s 1s/step - loss: 0.0788 - accuracy: 0.9776 - val_loss: 0.0453 - val_accuracy: 0.9922
Epoch 5/15
95/95 [==============================] - 121s 1s/step - loss: 0.0693 - accuracy: 0.9792 - val_loss: 0.0396 - val_accuracy: 0.9922
Epoch 6/15
95/95 [==============================] - 127s 1s/step - loss: 0.0664 - accuracy: 0.9786 - val_loss: 0.0369 - val_accuracy: 0.9909
Epoch 7/15
95/95 [==============================] - 127s 1s/step - loss: 0.0549 - accuracy: 0.9838 - val_loss: 0.0339 - val_accura

E:\New folder\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
